In [1]:
import json
import pandas as pd
import numpy as np
import os, sys
import re

# load data

In [2]:
# load all files in the folder
file = '.../phone_number_classified/crawl_data/data'
dirs = os.listdir(file)
print(len(dirs))
print(dirs[:2])

14
['data04.json', 'data09.json']


In [3]:
ls_data = []
for file in dirs:
    #print(file)
    keys_file = open(".../phone_number_classified/crawl_data/data/{}".format(file))
    keys = keys_file.read().encode('utf-8')
    keys_json = json.loads(keys)
    df = pd.DataFrame.from_dict(keys_json)
    ls_data.append(df)

In [4]:
df_data = pd.concat(ls_data)
df_data = df_data.dropna()
df_data.phone_number = df_data.phone_number.astype('str')
df_data.price = df_data.price.astype('str')
df_data.head(1)

,phone_number,price
0,0855.666.666,"Giá: 2,349,000,000"


# preprocessing data

In [5]:
def normlize_number_sequence(num_sequence):
    # convert tất cả những ký tự lớn hơn 2 khoảng trắng trởlên thành 1 khoảng trắng
    #num_sequence = re.sub(r'\s+', ' ', num_sequence)
    num_sequence = (num_sequence.lower()).strip()
    # xóa tất cả ký tự không phải là số
    sequence = re.sub(r'\D','', num_sequence)
    return  sequence.strip()
df_data['price'] = df_data.price.apply(normlize_number_sequence)
df_data['mobile_number'] = df_data.phone_number.apply(normlize_number_sequence)
df_data.head(1)

,phone_number,price,mobile_number
0,0855.666.666,2349000000,0855666666


In [6]:
df_data.price_1 = df_data.price.astype('float')
df_data = df_data.drop(['phone_number'], axis= 1)

/home/anhnvh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [7]:
#  create a dictionary, convert dau so di dong
convert_dau_so = {'162' : '32',
    '163' : '33',
    '164' : '34',
    '165' : '35',
    '166' : '36',
    '167' : '37',
    '168' : '38',
    '169' : '39',
    '120' : '70',
    '121' : '79',
    '122' : '77',
    '126' : '76',
    '128' : '78',
    '123' : '83',
    '124' : '84',
    '125' : '85',
    '127' : '81',
    '129' :'82',
    '199' : '59'}

# normalize mobile number
def norm_mobile(mobile_number):
    '''normalize mobile number (str)'''
    if mobile_number[0] == '0':
        mobile = mobile_number[1:]
    else:
        mobile = mobile_number
    if mobile[:3] in convert_dau_so.keys():
        mobile = convert_dau_so[mobile[:3]] + mobile[3:]
    return mobile
def count_char(mobile_number):
    '''count the character amount, before normalize mobile number'''
    return len(mobile_number)
df_data['mobile_number'] = df_data.mobile_number.astype('str').apply(norm_mobile)
df_data['length_of_mobile_number'] = df_data.mobile_number.astype('str').apply(count_char)

In [8]:
df_data = df_data[df_data.length_of_mobile_number == 9]
df_data = df_data.drop_duplicates(subset= ['mobile_number'])
df_data = df_data.dropna()
df_data.columns = ['price', 'mobile_number', 'length_of_mobile_number']
df_data.head(2)

,price,mobile_number,length_of_mobile_number
0,2349000000,855666666,9
1,302400000,359000000,9


In [9]:
df_data.price = df_data.price.astype('float')

1. new target
+ < 200.000 : 0
+ 200.000 - 499.000: 1
+ 500.000 - 999.000: 2
+ 1.000.000- 2.999.000: 3
+ 3.000.000-9.999.000: 4
+ 10.000.000 - 49.999.000: 5
+ \> 50.000.000: 6

In [10]:
# convert target 2
def convert_label(price):
    if price < 200000:
        return 0
    elif (price >= 200000) & (price < 500000):
        return 1
    elif (price >= 500000) & (price < 1500000):
        return 2
    elif (price >= 1500000) & (price < 10000000):
        return 3
    elif (price >= 10000000) & (price < 50000000):
        return 4
    elif (price >= 50000000):
        return 5
    else:
        pass
df_data['target'] = df_data.price.apply(convert_label)

In [11]:
df_data.target = df_data.target.astype('int')

In [12]:
df_data.target.value_counts()

2    269914
3    200012
0     49938
4     36198
1     25270
5     13704
Name: target, dtype: int64

In [13]:
df_data = df_data.reset_index()
df_data = df_data.drop(['index', 'price'], axis= 1)

In [14]:
df_data.head(1)

,mobile_number,length_of_mobile_number,target
0,855666666,9,5


In [15]:
df_data.columns

Index(['mobile_number', 'length_of_mobile_number', 'target'], dtype='object')

# split train test

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
idxs = df_data.index.values
targets = df_data.target.values
id_train, id_test, target_train, target_test = train_test_split(idxs, targets, test_size = 0.2)

In [18]:
df_train = df_data[df_data.index.isin(id_train)]
df_test = df_data[~df_data.index.isin(id_train)]

In [21]:
df_train = df_train.dropna()
df_train = df_train.reset_index()
df_train = df_train.drop(['index', 'length_of_mobile_number'], axis= 1)
df_train = df_train.reset_index()
df_train.columns = ['id_train', 'mobile_number', 'target']
print(df_train.shape)
df_test = df_test.dropna()
df_test = df_test.reset_index()
df_test = df_test.drop(['index', 'length_of_mobile_number'], axis= 1)
df_test = df_test.reset_index()
df_test.columns = ['id_test', 'mobile_number', 'target']
print(df_test.shape)

(476028, 3)
(119008, 3)


In [25]:
writer = pd.ExcelWriter('train_data.xlsx', engine='xlsxwriter')
df_train.to_excel(writer, 'Sheet1', encoding= 'utf-8', index= False)
writer.save()

In [26]:
writer = pd.ExcelWriter('test_data.xlsx', engine='xlsxwriter')
df_test.to_excel(writer, 'Sheet1', encoding= 'utf-8', index= False)
writer.save()

In [ ]:
# df_train = pd.ExcelFile('train_data.xlsx', encoding= 'utf-8').parse('Sheet1')

In [ ]:
# df_train